In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]

In [10]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [11]:
dependent=dataset[['Purchased']]

In [12]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [14]:
#from sklearn.preprocessing import StandardScaler
#sc=StandardScaler()
#x_train=sc.fit_transform(x_train)
#x_test=sc.transform(x_test)

In [15]:
from sklearn.naive_bayes import CategoricalNB
from sklearn. model_selection import GridSearchCV
param_grid = { 'force_alpha':[True,False], 
               'fit_prior':[True,False], 
               'class_prior':[None],
               'min_categories':[None]}
 
grid = GridSearchCV(CategoricalNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train,y_train)  
print("Best parameters found: ", grid.best_params_)
y_pred = grid.predict(x_test)
print("Predictions: ", y_pred)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters found:  {'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'min_categories': None}
Predictions:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0
 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

                   

In [53]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [55]:
print(cm)

[[82  3]
 [18 31]]


In [57]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [59]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.82      0.96      0.89        85
           1       0.91      0.63      0.75        49

    accuracy                           0.84       134
   macro avg       0.87      0.80      0.82       134
weighted avg       0.85      0.84      0.84       134



In [61]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'min_categories': None}: 0.8354758282828774


In [63]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.8955582232893157

In [23]:
table=pd.DataFrame.from_dict(re)

In [65]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_prior,param_fit_prior,param_force_alpha,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.050673,0.002403,0.029401,0.001311,None,True,True,None,"{'class_prior': None, 'fit_prior': True, 'forc...",0.726566,0.776838,0.673750,0.818941,NaN,NaN,NaN,1
1,0.042293,0.004871,0.026107,0.004146,None,True,False,None,"{'class_prior': None, 'fit_prior': True, 'forc...",0.726566,0.776838,0.673750,0.818941,NaN,NaN,NaN,1
2,0.053866,0.010356,0.035867,0.005287,None,False,True,None,"{'class_prior': None, 'fit_prior': False, 'for...",0.735616,0.828959,0.710831,0.869709,NaN,NaN,NaN,1
3,0.045591,0.003769,0.030342,0.006331,None,False,False,None,"{'class_prior': None, 'fit_prior': False, 'for...",0.735616,0.828959,0.710831,0.869709,NaN,NaN,NaN,1
